In [ ]:
!pip install gradio

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import gradio as gr
import asyncio

# 設定裝置為 GPU 或 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入訓練好的模型和 tokenizer
model_path = "NetflixGPT-english"  # 修改為你訓練模型的儲存路徑
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to(device)
model.eval()

In [ ]:
# 定義生成描述的函數
def generate_description(title, max_length=100):
    # 構建輸入文本
    input_text = f"<|startoftext|>Title: {title} <|sep|>Description:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # 使用模型進行生成
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )

    # 直接解碼從模型生成的部分
    generated_text = tokenizer.decode(output[0][input_ids.shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

# 定義逐字顯示的異步函數
async def display_text(title):
    description = generate_description(title)
    displayed_text = ""
    for char in description:
        displayed_text += char
        await asyncio.sleep(0.05)  # 控制每個字顯示的間隔時間
        yield displayed_text




In [ ]:

# 使用 Gradio 創建介面
iface = gr.Interface(
    fn=display_text,
    inputs=gr.Dropdown(choices=["Stranger Things", "Breaking Bad", "The Crown"], label="Select a Title"),
    outputs=gr.Textbox(label="Generated Description"),
    title="Netflix Title Description Generator",
    description="Enter a Netflix title, and the model will generate a description."
)
# 啟動 Gradio 介面並指定埠
iface.launch()